In [1]:
import json
import pandas as pd
pd.set_option('display.max_rows', None)
import os
import numpy as np
import csv
import random

In [2]:
os.chdir("C:/Users/Sankar/SDN_Project/JSON/JSON")
files=os.listdir('C:/Users/Sankar/SDN_Project/JSON/JSON/')
data=[]
for file in files:    
    with open(file) as f:
        df = json.load(f)
        data.append(df)

In [3]:
def getSourceDestination(data):
    if 'dnsname' in str(data):
        src_net_key = 'ietf-acldns:src-dnsname'
        dest_net_key = 'ietf-acldns:dst-dnsname'
    elif 'network' in str(data):
        src_net_key = 'source-ipv4-network'
        dest_net_key = 'destination-ipv4-network'
    else:
        src_net_key = '*'
        dest_net_key = '*'
    return data.get(src_net_key, '*'), data.get(dest_net_key, '*'), data.get('protocol', '*')
    

In [4]:
final_list = []
for i in range(len(data)):
    access_json=data[i].get('ietf-access-control-list:access-lists').get('acl')    
    for acl in access_json:
        aces = acl['aces']['ace']
        for ace in aces:
            final_row = {
            'sEth': '*',
            'dEth': '*',
            'typEth': '*',
            }
            if 'ethertype' in str(ace['matches']):
                final_row['sEth'] = ace['matches']['eth'].get('source-mac-address', '*')
                final_row['dEth'] = ace['matches']['eth'].get('destination-mac-address', '*')
                final_row['typEth'] = ace['matches']['eth'].get('ethertype', '*')
            if 'ipv4' in str(ace['matches']):
                source, dest, proto = getSourceDestination(ace['matches']['ipv4'])
                final_row['Source'] = source
                final_row['Destination'] = dest
                final_row['proto'] = proto
            elif 'ipv6' in str(ace['matches']):
                source, dest, proto = getSourceDestination(ace['matches']['ipv6'])
                final_row['Source'] = source
                final_row['Destination'] = dest
                final_row['proto'] = proto
            else:
                final_row['Source'] = '*'
                final_row['Destination'] = '*'
                final_row['proto'] = '*'
            if 'ipv4' in str(ace['matches']):
                source, dest, proto = getSourceDestination(ace['matches']['ipv4'])
                if proto in [1,6,17] :
                    final_row['typEth']='0x0800'
            if 'icmp' in str(ace['matches']):
                final_row['type'] = ace['matches']['icmp'].get('type', '*')
                final_row['code'] = ace['matches']['icmp'].get('code', '*')
            else:
                final_row['type'] = '*' 
                final_row['code'] = '*'   
            if 'source-port' in str(ace['matches']):
                final_row['sPort'] = ace['matches']['udp']['source-port'].get('port', '*') if 'udp' in str(ace['matches']) else ace['matches']['tcp']['source-port'].get('port', '*')
            else:
                final_row['sPort'] = '*'
            if 'destination-port' in str(ace['matches']):
                final_row['dPort'] = ace['matches']['udp']['destination-port'].get('port', '*') if 'udp' in str(ace['matches']) else ace['matches']['tcp']['destination-port'].get('port', '*')
            else:
                final_row['dPort'] = '*'
            final_row['priority'] = '*'
            final_row['action'] = 'forward' if ace['actions'].get('forwarding', '') == 'accept' else '*'
            if final_row['Source'] == '*' and final_row['Destination'] != '*':
                final_row['sEth'] = '9e:8d:de:80:29:28'  
            if final_row['Destination'] == '*' and final_row['Source'] != '*':
                final_row['dEth'] = '9e:8d:de:80:29:28'     
            final_list.append(final_row)

df = pd.DataFrame(final_list)
df.head()


,sEth,dEth,typEth,Source,Destination,proto,type,code,sPort,dPort,priority,action
0,9e:8d:de:80:29:28,*,0x0800,*,dcape-na.amazon.com,6,*,*,*,443,*,forward
1,9e:8d:de:80:29:28,*,0x0800,*,softwareupdates.amazon.com,6,*,*,*,443,*,forward
2,9e:8d:de:80:29:28,*,0x0800,*,3.north-america.pool.ntp.org,17,*,*,*,123,*,forward
3,9e:8d:de:80:29:28,*,*,*,224.0.0.22/32,2,*,*,*,*,*,forward
4,9e:8d:de:80:29:28,*,0x0800,*,239.255.255.250/32,17,*,*,*,1900,*,forward


In [5]:
df.loc[len(df.index)] = ['*','*','*','*','*','*','*','*','*','*','*','deny']

In [6]:
df.tail()

,sEth,dEth,typEth,Source,Destination,proto,type,code,sPort,dPort,priority,action
1021,*,9e:8d:de:80:29:28,0x0800,scalews.withings.net,*,6,*,*,443,*,*,forward
1022,*,*,0x0800,*,*,17,*,*,67,*,*,forward
1023,*,9e:8d:de:80:29:28,0x0800,scalews.withings.net,*,6,*,*,80,*,*,forward
1024,*,*,0x888e,*,*,*,*,*,*,*,*,forward
1025,*,*,*,*,*,*,*,*,*,*,*,deny


In [7]:
df.to_csv('C:/Users/Sankar/SDN_Project/ACL_All_IOT_Devices.csv')

In [8]:
column = ['sEth','dEth','typEth','Source','Destination','proto','sPort','dPort','priority','action']

In [9]:
df2 = df[column]
df1 =df2.drop_duplicates()

In [10]:
df1.shape

(698, 10)

In [11]:
df1.to_csv('C:/Users/Sankar/SDN_Project/Project_data_final.csv')

In [12]:
#raw_df = pd.read_excel("C:/Users/Sankar/SDN_Project/Project_data_final.xlsx")

In [13]:
#type(df1)

In [14]:
#raw_df=raw_df.drop('priority', axis=1)
#raw_df.to_csv('C:/Users/Sankar/SDN_Project/Project_data_final_converted.csv',index=None,header=True)

In [15]:
##import csv 
##csv_file = 'C:/Users/Sankar/SDN_Project/Project_data_final_converted.csv' 
##txt_file = 'C:/Users/Sankar/SDN_Project/Project_data_final_text.txt'
##with open(txt_file, "w") as my_output_file:
##   with open(csv_file, "r") as my_input_file:
##        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
##    my_output_file.close()*/

In [16]:
raw_df2 = pd.read_csv("C:/Users/Sankar/SDN_Project/Project_data_final.csv")

In [17]:
raw_df2['Source'] = raw_df2['Source'].str.replace("/32", "")
raw_df2['Destination'] = raw_df2['Destination'].str.replace("/32", "")

In [18]:
raw_df2['Source1']=raw_df2['Source']
raw_df2['Destination1']=raw_df2['Destination']

In [19]:
raw_df2.head()

,Unnamed: 0,sEth,dEth,typEth,Source,Destination,proto,sPort,dPort,priority,action,Source1,Destination1
0,0,9e:8d:de:80:29:28,*,0x0800,*,dcape-na.amazon.com,6,*,443,*,forward,*,dcape-na.amazon.com
1,1,9e:8d:de:80:29:28,*,0x0800,*,softwareupdates.amazon.com,6,*,443,*,forward,*,softwareupdates.amazon.com
2,2,9e:8d:de:80:29:28,*,0x0800,*,3.north-america.pool.ntp.org,17,*,123,*,forward,*,3.north-america.pool.ntp.org
3,3,9e:8d:de:80:29:28,*,*,*,224.0.0.22,2,*,*,*,forward,*,224.0.0.22
4,4,9e:8d:de:80:29:28,*,0x0800,*,239.255.255.250,17,*,1900,*,forward,*,239.255.255.250


In [20]:
x = raw_df2['Source'].unique()
y = raw_df2['Destination'].unique()

In [21]:

z=np.append(y,x)

In [22]:
z =np.unique(z)
len(z)

254

In [23]:
z = z[z!='*']

In [24]:
len(z)

253

In [25]:
unique_values = z.tolist()

In [26]:
temp_dict ={}
for val in unique_values:
    temp_dict[val] = ".".join(map(str, (random.randint(0, 255) 
                        for _ in range(4))))

In [27]:
raw_df2["Source1"].replace(temp_dict, inplace=True)
raw_df2["Destination1"].replace(temp_dict, inplace=True)

In [28]:
raw_df2.head()

,Unnamed: 0,sEth,dEth,typEth,Source,Destination,proto,sPort,dPort,priority,action,Source1,Destination1
0,0,9e:8d:de:80:29:28,*,0x0800,*,dcape-na.amazon.com,6,*,443,*,forward,*,241.167.203.158
1,1,9e:8d:de:80:29:28,*,0x0800,*,softwareupdates.amazon.com,6,*,443,*,forward,*,128.166.143.111
2,2,9e:8d:de:80:29:28,*,0x0800,*,3.north-america.pool.ntp.org,17,*,123,*,forward,*,225.42.189.219
3,3,9e:8d:de:80:29:28,*,*,*,224.0.0.22,2,*,*,*,forward,*,227.132.136.119
4,4,9e:8d:de:80:29:28,*,0x0800,*,239.255.255.250,17,*,1900,*,forward,*,10.239.139.155


In [29]:
temp_dict_1 ={}
for val in raw_df2['Source1'].tolist():
    if(val=='*'):
        temp_dict_1[val] ='*'
    else:
        temp_dict_1[val]=''.join([hex(int(x)+256)[3:] for x in val.split('.')])
                        

In [30]:
temp_dict_2 ={}
for val1 in raw_df2['Destination1'].tolist():
    if(val1=='*'):
        temp_dict_2[val1] ='*'
    else:
        temp_dict_2[val1]=''.join([hex(int(x)+256)[3:] for x in val1.split('.')])

In [31]:
raw_df2["Source1"].replace(temp_dict_1, inplace=True)
raw_df2["Destination1"].replace(temp_dict_2, inplace=True)

In [32]:
raw_df2.head()

,Unnamed: 0,sEth,dEth,typEth,Source,Destination,proto,sPort,dPort,priority,action,Source1,Destination1
0,0,9e:8d:de:80:29:28,*,0x0800,*,dcape-na.amazon.com,6,*,443,*,forward,*,f1a7cb9e
1,1,9e:8d:de:80:29:28,*,0x0800,*,softwareupdates.amazon.com,6,*,443,*,forward,*,80a68f6f
2,2,9e:8d:de:80:29:28,*,0x0800,*,3.north-america.pool.ntp.org,17,*,123,*,forward,*,e12abddb
3,3,9e:8d:de:80:29:28,*,*,*,224.0.0.22,2,*,*,*,forward,*,e3848877
4,4,9e:8d:de:80:29:28,*,0x0800,*,239.255.255.250,17,*,1900,*,forward,*,0aef8b9b


In [33]:
raw_df2['Source']=raw_df2['Source1']
raw_df2['Destination']=raw_df2['Destination1']

In [34]:
raw_df2 = raw_df2.drop(['Source1', 'Destination1'], axis=1)

In [35]:
raw_df2.tail()

,Unnamed: 0,sEth,dEth,typEth,Source,Destination,proto,sPort,dPort,priority,action
693,1014,9e:8d:de:80:29:28,*,0x0800,*,2f1824b7,6,*,443,*,forward
694,1018,9e:8d:de:80:29:28,*,0x0800,*,2f1824b7,6,*,80,*,forward
695,1021,*,9e:8d:de:80:29:28,0x0800,2f1824b7,*,6,443,*,*,forward
696,1023,*,9e:8d:de:80:29:28,0x0800,2f1824b7,*,6,80,*,*,forward
697,1025,*,*,*,*,*,*,*,*,*,deny


In [36]:
column_list = ['action','sEth','dEth','typEth','Source', 'Destination','proto','sPort',  'dPort']

In [37]:
raw_df2 = raw_df2[column_list]

In [38]:
len(raw_df2)

698

In [39]:
raw_df2['Priority'] = range(1, 1+len(raw_df2))

In [40]:
raw_df2.loc[raw_df2['action']=='deny','Priority']=''

In [41]:
raw_df2['MAC_Address']='00:00:00:00:00:00'
raw_df2['Output_port']=1
raw_df2.loc[raw_df2['action']=='deny','MAC_Address']=''
raw_df2.loc[raw_df2['action']=='deny','Output_port']=''
raw_df2.tail()

,action,sEth,dEth,typEth,Source,Destination,proto,sPort,dPort,Priority,MAC_Address,Output_port
693,forward,9e:8d:de:80:29:28,*,0x0800,*,2f1824b7,6,*,443,694,00:00:00:00:00:00,1
694,forward,9e:8d:de:80:29:28,*,0x0800,*,2f1824b7,6,*,80,695,00:00:00:00:00:00,1
695,forward,*,9e:8d:de:80:29:28,0x0800,2f1824b7,*,6,443,*,696,00:00:00:00:00:00,1
696,forward,*,9e:8d:de:80:29:28,0x0800,2f1824b7,*,6,80,*,697,00:00:00:00:00:00,1
697,deny,*,*,*,*,*,*,*,*,,,


In [42]:
raw_df2.to_csv('C:/Users/Sankar/SDN_Project/Project_data_withoutBDD_original.csv',index=None,header=True)

In [43]:
raw_df3=pd.read_csv('C:/Users/Sankar/SDN_Project/Project_data_withoutBDD_original.csv')

In [44]:
raw_df3['sEth'] = raw_df3.sEth.str.replace('[:]', '')
raw_df3['dEth'] = raw_df3.dEth.str.replace('[:]', '')
raw_df3['Source'] = raw_df3.Source.str.replace('[.]', '')
raw_df3['Destination'] = raw_df3.Destination.str.replace('[.]', '')
raw_df3.head()

,action,sEth,dEth,typEth,Source,Destination,proto,sPort,dPort,Priority,MAC_Address,Output_port
0,forward,9e8dde802928,*,0x0800,*,f1a7cb9e,6,*,443,1.0,00:00:00:00:00:00,1.0
1,forward,9e8dde802928,*,0x0800,*,80a68f6f,6,*,443,2.0,00:00:00:00:00:00,1.0
2,forward,9e8dde802928,*,0x0800,*,e12abddb,17,*,123,3.0,00:00:00:00:00:00,1.0
3,forward,9e8dde802928,*,*,*,e3848877,2,*,*,4.0,00:00:00:00:00:00,1.0
4,forward,9e8dde802928,*,0x0800,*,0aef8b9b,17,*,1900,5.0,00:00:00:00:00:00,1.0


In [45]:
raw_df3.loc[raw_df3['action']=='deny','Priority']=0
raw_df3.Priority = raw_df3.Priority.astype('int64')

In [46]:
raw_df3.loc[raw_df3['action']=='deny','Output_port']=0
raw_df3.Output_port = raw_df3.Output_port.astype('int64')
raw_df3.loc[raw_df3['action']=='deny','Output_port']=''
raw_df3.tail()

,action,sEth,dEth,typEth,Source,Destination,proto,sPort,dPort,Priority,MAC_Address,Output_port
693,forward,9e8dde802928,*,0x0800,*,2f1824b7,6,*,443,694,00:00:00:00:00:00,1
694,forward,9e8dde802928,*,0x0800,*,2f1824b7,6,*,80,695,00:00:00:00:00:00,1
695,forward,*,9e8dde802928,0x0800,2f1824b7,*,6,443,*,696,00:00:00:00:00:00,1
696,forward,*,9e8dde802928,0x0800,2f1824b7,*,6,80,*,697,00:00:00:00:00:00,1
697,deny,*,*,*,*,*,*,*,*,0,NaN,


In [47]:
raw_df3.loc[raw_df3['action']=='deny','MAC_Address']=''

In [48]:
raw_df3.loc[raw_df3['action']=='deny','Priority']=''

In [49]:
raw_df3.tail()

,action,sEth,dEth,typEth,Source,Destination,proto,sPort,dPort,Priority,MAC_Address,Output_port
693,forward,9e8dde802928,*,0x0800,*,2f1824b7,6,*,443,694,00:00:00:00:00:00,1
694,forward,9e8dde802928,*,0x0800,*,2f1824b7,6,*,80,695,00:00:00:00:00:00,1
695,forward,*,9e8dde802928,0x0800,2f1824b7,*,6,443,*,696,00:00:00:00:00:00,1
696,forward,*,9e8dde802928,0x0800,2f1824b7,*,6,80,*,697,00:00:00:00:00:00,1
697,deny,*,*,*,*,*,*,*,*,,,


In [50]:
raw_df3.to_csv('C:/Users/Sankar/SDN_Project/Project_data_withoutBDD_temp.csv',index=None,header=True)

In [51]:
import csv
csv_file = 'C:/Users/Sankar/SDN_Project/Project_data_withoutBDD_temp.csv' 
txt_file = 'C:/Users/Sankar/SDN_Project/Project_data_withoutBDD_text1.txt'
with open(txt_file, "w") as my_output_file:
    with open(csv_file, "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
        my_output_file.close()

In [52]:
raw_df3=pd.read_csv('C:/Users/Sankar/SDN_Project/Project_data_withoutBDD_temp.csv')

In [53]:
raw_df3.head()

,action,sEth,dEth,typEth,Source,Destination,proto,sPort,dPort,Priority,MAC_Address,Output_port
0,forward,9e8dde802928,*,0x0800,*,f1a7cb9e,6,*,443,1.0,00:00:00:00:00:00,1.0
1,forward,9e8dde802928,*,0x0800,*,80a68f6f,6,*,443,2.0,00:00:00:00:00:00,1.0
2,forward,9e8dde802928,*,0x0800,*,e12abddb,17,*,123,3.0,00:00:00:00:00:00,1.0
3,forward,9e8dde802928,*,*,*,e3848877,2,*,*,4.0,00:00:00:00:00:00,1.0
4,forward,9e8dde802928,*,0x0800,*,0aef8b9b,17,*,1900,5.0,00:00:00:00:00:00,1.0


In [54]:
raw_df4 = raw_df3

In [55]:
raw_df4['sEth'].unique()

array(['9e8dde802928', '*'], dtype=object)

In [56]:
raw_df4.loc[raw_df4['sEth']!='*','sEth']= '0x' + raw_df4['sEth']+'&&&0x0fff'

In [57]:
raw_df4.loc[raw_df4['sEth']=='*','sEth']= '0x0&&&0x0'

In [58]:
raw_df4.loc[raw_df4['dEth']!='*','dEth']= '0x' + raw_df4['dEth']+'&&&0x0fff'
raw_df4.loc[raw_df4['dEth']=='*','dEth']= '0x0&&&0x0'
raw_df4.loc[raw_df4['typEth']!='*','typEth']= raw_df4['typEth']+'&&&0x0fff'
raw_df4.loc[raw_df4['typEth']=='*','typEth']= '0x0&&&0x0'
raw_df4.loc[raw_df4['Source']!='*','Source']= '0x' + raw_df4['Source']+'&&&0x0fff'
raw_df4.loc[raw_df4['Source']=='*','Source']= '0x0&&&0x0'
raw_df4.loc[raw_df4['Destination']!='*','Destination']= '0x' + raw_df4['Destination']+'&&&0x0fff'
raw_df4.loc[raw_df4['Destination']=='*','Destination']= '0x0&&&0x0'
raw_df4.loc[raw_df4['proto']!='*','proto']= '0x' + raw_df4['proto']+'&&&0x0ff'
raw_df4.loc[raw_df4['proto']=='*','proto']= '0x0&&&0x0'
raw_df4.loc[raw_df4['sPort']!='*','sPort']= '0x' + raw_df4['sPort']+'&&&0x0ff'
raw_df4.loc[raw_df4['sPort']=='*','sPort']= '0x0&&&0x0'
raw_df4.loc[raw_df4['dPort']!='*','dPort']= '0x' + raw_df4['dPort']+'&&&0x0ff'
raw_df4.loc[raw_df4['dPort']=='*','dPort']= '0x0&&&0x0'

In [59]:
raw_df4.loc[raw_df4['action']=='deny','Priority']=0
raw_df4.Priority = raw_df4.Priority.astype('int64')
raw_df4.loc[raw_df4['action']=='deny','Output_port']=0
raw_df4.Output_port = raw_df4.Output_port.astype('int64')
raw_df4.loc[raw_df4['action']=='deny','Output_port']=''
raw_df4.loc[raw_df4['action']=='deny','MAC_Address']=''
raw_df4.loc[raw_df4['action']=='deny','Priority']=''
raw_df4.tail()

,action,sEth,dEth,typEth,Source,Destination,proto,sPort,dPort,Priority,MAC_Address,Output_port
693,forward,0x9e8dde802928&&&0x0fff,0x0&&&0x0,0x0800&&&0x0fff,0x0&&&0x0,0x2f1824b7&&&0x0fff,0x6&&&0x0ff,0x0&&&0x0,0x443&&&0x0ff,694,00:00:00:00:00:00,1
694,forward,0x9e8dde802928&&&0x0fff,0x0&&&0x0,0x0800&&&0x0fff,0x0&&&0x0,0x2f1824b7&&&0x0fff,0x6&&&0x0ff,0x0&&&0x0,0x80&&&0x0ff,695,00:00:00:00:00:00,1
695,forward,0x0&&&0x0,0x9e8dde802928&&&0x0fff,0x0800&&&0x0fff,0x2f1824b7&&&0x0fff,0x0&&&0x0,0x6&&&0x0ff,0x443&&&0x0ff,0x0&&&0x0,696,00:00:00:00:00:00,1
696,forward,0x0&&&0x0,0x9e8dde802928&&&0x0fff,0x0800&&&0x0fff,0x2f1824b7&&&0x0fff,0x0&&&0x0,0x6&&&0x0ff,0x80&&&0x0ff,0x0&&&0x0,697,00:00:00:00:00:00,1
697,deny,0x0&&&0x0,0x0&&&0x0,0x0&&&0x0,0x0&&&0x0,0x0&&&0x0,0x0&&&0x0,0x0&&&0x0,0x0&&&0x0,,,


In [60]:
raw_df4['new_column'] = "table_add data"
raw_df4['new_column2'] ="=>"
column_list = ['new_column','action','sEth','dEth','typEth','Source', 'Destination','proto','sPort', 'dPort','new_column2','MAC_Address','Output_port','Priority']
raw_df4 = raw_df4[column_list]
raw_df4.to_csv('C:/Users/Sankar/SDN_Project/Project_data_withoutBDD_final.csv',index=None,header=True)
import csv
csv_file = 'C:/Users/Sankar/SDN_Project/Project_data_withoutBDD_final.csv' 
txt_file = 'C:/Users/Sankar/SDN_Project/Project_data_withoutBDD_text_final1.txt'
with open(txt_file, "w") as my_output_file:
    with open(csv_file, "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
        my_output_file.close()

In [61]:
#str(state_counter)
#final_data.append({column : val, "state": "state-"+str(state_counter)})
#state_counter = state_counter + 1
#final_df = pd.DataFrame.from_dict(final_data)

In [62]:
#import random

#ip = ".".join(map(str, (random.randint(0, 255) 
#                        for _ in range(4))))